Generic Types, Traits and Lifetime
---

Every programming language has tools for effectively handling the duplication of  
concepts. In Rust, one such tool is generics: abstract stand-ins for concrete  
types or other properties. We can express the behavior of generics or how they  
relate to other generics without knowing what will be in their place when  
compiling and running the code.  

Functions can take parameters of some generic type, instead of a concrete type  
like i32 or String, in the same way a function takes parameters with unknown  
values to run the same code on multiple concrete values. In fact, we’ve already  
used generics in Chapter 6 with Option<T>, Chapter 8 with Vec<T> and  
HashMap<K, V>, and Chapter 9 with Result<T, E>. In this chapter, you’ll explore  
how to define your own types, functions, and methods with generics!  

First, we’ll review how to extract a function to reduce code duplication. We’ll  
then use the same technique to make a generic function from two functions that  
differ only in the types of their parameters. We’ll also explain how to use  
generic types in struct and enum definitions.  

Then you’ll learn how to use traits to define behavior in a generic way. You  
can combine traits with generic types to constrain a generic type to accept only  
those types that have a particular behavior, as opposed to just any type.  

Finally, we’ll discuss lifetimes: a variety of generics that give the compiler  
information about how references relate to each other. Lifetimes allow us to  
give the compiler enough information about borrowed values so that it can ensure  
references will be valid in more situations than it could without our help.  

### Removing Duplication by Extracting a Function

Generics allow us to replace specific types with a placeholder that represents  
multiple types to remove code duplication. Before diving into generics syntax,  
then, let’s first look at how to remove duplication in a way that doesn’t  
involve generic types by extracting a function that replaces specific values  
with a placeholder that represents multiple values. Then we’ll apply the same  
technique to extract a generic function! By looking at how to recognize  
duplicated code you can extract into a function, you’ll start to recognize  
duplicated code that can use generics.

We begin with the short program in Listing 10-1 that finds the largest number in  
a list.

> Filename: src/main.rs

In [2]:
{   // begin closure
    let number_list = vec![34, 50, 25, 100, 65]; // number_list comes into scope and
                                                 // gets ownership of the vector, so
                                                 // is responsible for releasing it
                                                 // - `number_list` is stored on the stack
                                                 // it contains meta data about the vector
                                                 // wrt to length and capacity of the ACTUAL
                                                 // VALUE on the HEAP

    let mut largest = &number_list[0];  // largest comes into scope
                                        // we don't take ownership of the vector,
                                        // we borrow a reference to it
                                        // largest stores this reference on the stack

    for number in &number_list {    // looping over an immutable reference to &number_list
                                    // - by using a reference we AVOID loop taking ownership of
                                    // the number_list vector, so we can use it later (println!)
                                    // - the loop will create an immutable reference `number` to
                                    // each element in the vector
        if number > largest {       // we compare `number` to `largest`
                                    // - because both are references, Rust automatically
                                    // dereferences each value for comparison thanks to `deref` trait
            largest = number;       // if current `number` is larger than `largest`
                                    // - we update the mutable reference `largest` to point to
                                    // the current `number` reference
                                    // - we DO NOT take OWNERSHIP of the VALUE, only update REFERENCE
        }
    }

    println!("The largest number is {}", largest);

}   // exit closure
    // largest goes out of scope, and are released : it is a reference so no heap changes
    // number_list goes out of scope, and is released from memory
    // - the vector meta data wrt to length and capacity is released from the stack
    // - Rust automatically calls the `drop` function to release the VECTOR's VALUE
    // from the HEAP

The largest number is 100


()

> Listing 10-1: Finding the largest number in a list of numbers

We store a list of integers in the variable `number_list` and place a reference to  
the first number in the list in a variable named `largest`. We then iterate  
through all the numbers in the list, and if the current number is greater than  
the number stored in `largest`, replace the reference in that variable. However,  
if the current number is less than or equal to the largest number seen so far,  
the variable doesn’t change, and the code moves on to the next number in the  
list. After considering all the numbers in the list, `largest` should refer to  
the largest number, which in this case is 100.  

We've now been tasked with finding the largest number in two different lists of  
numbers. To do so, we can choose to duplicate the code in Listing 10-1 and use  
the same logic at two different places in the program, as shown in Listing 10-2.  

> Filename: src/main.rs

In [4]:
{
    let number_list = vec![34, 50, 25, 100, 65];

    let mut largest = &number_list[0];

    for number in &number_list {
        if number > largest {
            largest = number;
        }
    }

    println!("The largest number is {}", largest);

    let number_list = vec![102, 34, 6000, 89, 54, 2, 43, 8];

    let mut largest = &number_list[0];

    for number in &number_list{
        if number > largest {
            largest = number;
        }
    }

    println!("The largest number is {}", largest);
}

The largest number is 100
The largest number is 6000


()

> Listing 10-2: Code to find the largest number in two lists of numbers

Although this code works, duplicating code is tedious and error prone. We also  
have to remember to **`update the code in multiple places`** when we want to  
change it.

To eliminate this duplication, we’ll create an abstraction by defining a  
function that operates on any list of integers passed in a parameter. This  
solution makes our code clearer and lets us express the concept of finding the  
largest number in a list abstractly.

In Listing 10-3, we extract the code that finds the largest number into a  
function named largest. Then we call the function to find the largest number  
in the two lists from Listing 10-2. We could also use the function on any other  
list of i32 values we might have in the future.

> Filename: src/main.rs



In [5]:
fn largest(list: &[i32])-> &i32 {
    let mut largest = &list[0];

    for number in list {
        if number > largest {
            largest = number;
        }
    }

    largest
}

{
    let number_list = vec![34, 50, 25, 100, 65];
    let result = largest(&number_list);
    println!("The largest number is {}", result);

    let number_list = vec![102, 34, 6000, 89, 54, 2, 43, 8];
    let result = largest(&number_list);
    println!("The largest number is {}", result);
}

The largest number is 100
The largest number is 6000


()

> Listing 10-3: Abstracted code to find the largest number in two lists

The largest function has a parameter called list, which represents any concrete  
slice of i32 values we might pass into the function. As a result, when we call  
the function, the code runs on the specific values that we pass in.

